In [1]:
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine

from sqlalchemy import Integer, String, Numeric, SmallInteger

In [2]:
load_dotenv()

DB_CONFIG = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "database": os.getenv("DB_NAME"),
    "port": os.getenv("DB_PORT")
}

engine = create_engine(f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

## 🏆 **Average Points per Team**

```sql
average_points_per_team (
    id               SERIAL PRIMARY KEY,
    team_id          INTEGER NOT NULL,
    team_name        VARCHAR(50) NOT NULL,
    average_points   DECIMAL(3,2) NOT NULL
);

In [21]:
df = pd.read_json('../analysis/average_points/average_points_per_team.json')
df = df.rename(columns={
    'TeamID': 'team_id',
    'TeamName': 'team_name',
    'AveragePoints': 'average_points'
})
df.head()

,team_id,team_name,average_points
0,3,Кёльн,"[1.24, 1.28, 1.49, 0.79, 1.83, 1.08, 1.08, 1.5..."
1,5,Милан,"[1.38, 1.67, 1.68, 1.81, 1.73, 1.76, 2.02, 2.0..."
2,11,Арсенал,"[2.07, 1.78, 2.05, 1.77, 1.95, 1.69, 1.78, 1.8..."
3,12,Рома,"[1.66, 1.83, 2.13, 1.9, 1.69, 1.76, 1.77, 1.8,..."
4,13,Атлетико Мадрид,"[1.91, 2.14, 2.03, 2.0, 2.02, 1.78, 2.04, 1.69..."


In [23]:
df = df.explode('average_points').reset_index(drop=True)
df.head()

,team_id,team_name,average_points
0,3,Кёльн,1.24
1,3,Кёльн,1.28
2,3,Кёльн,1.49
3,3,Кёльн,0.79
4,3,Кёльн,1.83


In [ ]:
df.to_sql('average_points_per_team', 
          engine, 
          dtype={
              'team_id': Integer(),
              'team_name': String(50),
              'average_points': Numeric(3,2)
          },
          if_exists='append',
          index=False)

563

## 🌏 **Legionnaires per Team**

```sql
legionnaires_per_team (
    team_id   INTEGER NOT NULL CHECK (team_id > 0),
    year      SMALLINT NOT NULL CHECK (year BETWEEN 1900 AND 2100),
    legioners SMALLINT NOT NULL CHECK (legioners >= 0),
    PRIMARY KEY (team_id, year)
);

In [5]:
df = pd.read_json('../analysis/legionnaires/legionnaires_per_team.json')
df = df.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'Legioners': 'legioners'
})
df.head()

,team_id,year,legioners
0,3,2014,22
1,3,2015,17
2,3,2016,23
3,3,2017,28
4,3,2018,20


In [9]:
df.to_sql('legionnaires_per_team', 
          engine, 
          dtype={
              'team_id': Integer(),
              'year': SmallInteger(),
              'legioners': SmallInteger()
          },
          if_exists='append',
          index=False)

896

## 👕 **Clubs and National Players**

```sql
clubs_and_national_players (
    year                  SMALLINT NOT NULL CHECK (year BETWEEN 1900 AND 2100),
    team_id               INTEGER NOT NULL,
    players_in_national_team  SMALLINT NOT NULL CHECK (players_in_national_team >= 0),
    PRIMARY KEY (year, team_id)
);

In [11]:
df = pd.read_json('../analysis/players_in_national_teams/clubs_and_national_players.json')
df = df.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'PlayersInNationalTeam': 'players_in_national_team'
})
df.head()

,year,team_id,players_in_national_team
0,2014,3,9
1,2015,3,8
2,2016,3,5
3,2017,3,4
4,2018,3,3


In [12]:
df.to_sql('clubs_and_national_players', 
          engine, 
          dtype={
              'team_id': Integer(),
              'year': SmallInteger(),
              'players_in_national_team': SmallInteger()
          },
          if_exists='append',
          index=False)

896

## 📊👥 **Average Age per Team**

```sql
average_age_per_team (
    team_id      INTEGER NOT NULL,
    year         SMALLINT NOT NULL CHECK (year BETWEEN 2000 AND 2100),
    average_age  DECIMAL(3,1) NOT NULL CHECK (average_age >= 0),
    PRIMARY KEY (team_id, year)
);

In [14]:
df = pd.read_json('../analysis/average_age/average_age_per_team.json')
df = df.rename(columns={
    'TeamID': 'team_id',
    'Year': 'year',
    'AverageAge': 'average_age'
})
df.head()

,team_id,year,average_age
0,1002,2014,21.5
1,1002,2015,21.6
2,1002,2016,21.0
3,1002,2017,21.3
4,1002,2018,22.4


In [15]:
df.to_sql('average_age_per_team', 
          engine, 
          dtype={
              'team_id': Integer(),
              'year': SmallInteger(),
              'average_age': SmallInteger()
          },
          if_exists='append',
          index=False)

313